In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from tqdm import tqdm

directory_path_dataset = "/content/drive/MyDrive/disertatie/dataset"
directory_path_dataset_to_save = "/content/drive/MyDrive/disertatie/translate/NLLB_TRAINED"
name_of_translation = "nllb.txt"
file_to_translate_name = "ar.txt"
source_language = "arb_Arab"
target_language = "ron_Latn"

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained(
    "facebook/nllb-200-distilled-600M", src_lang=source_language)

model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/disertatie/FilteredDatasets/Eliminated_dialects/NLLB/checkpoint-12000", ) #/content/drive/MyDrive/disertatie/FilteredDatasets/Eliminated_dialects/NLLB/checkpoint-12000 facebook/nllb-200-distilled-600M

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

In [ ]:
device

device(type='cuda')

In [ ]:
if not os.path.exists(directory_path_dataset_to_save):
    os.makedirs(directory_path_dataset_to_save)

In [ ]:
def list_directories(directory):
    entries = os.listdir(directory)

    directories = [directory+"/"+entry for entry in entries if os.path.isdir(os.path.join(directory, entry))]

    print("Directories in", directory, ":")
    return directories

def create_all_missing_folders(directories_list):
    for folder in directories_list:
        folder_path_to_create = folder.split(directory_path_dataset)[1]
        if not os.path.exists(directory_path_dataset_to_save+folder_path_to_create):
            os.makedirs(directory_path_dataset_to_save+folder_path_to_create)

def create_translation_for_file(lines_to_translate, file_to_save_to):
    translated_lines = []

    length_lines = len(lines_to_translate)
    for line in tqdm(lines_to_translate):
        inputs = tokenizer(line, return_tensors="pt").to(device)
        translated_tokens = model.generate(
            **inputs, forced_bos_token_id=tokenizer.convert_tokens_to_ids(target_language), max_length=1024
        )
        line_translated = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
        translated_lines.append(line_translated)

    f = open(file_to_save_to, encoding="utf-8", mode='w')
    for i, line in enumerate(translated_lines):
        f.write(line)
        if i != length_lines-1:
            f.write("\n")
    f.close()
    print("The translation is saved at ", file_to_save_to)


def read_files_and_create_translations(directories_list):
    for folder in directories_list:
        folder_path_to_save = directory_path_dataset_to_save+folder.split(directory_path_dataset)[1]
        file_to_read = folder+"/"+file_to_translate_name
        print("Creating translations for ", file_to_read)
        f = open(file_to_read, encoding="utf-8")
        all_lines = f.readlines()
        all_lines_split = [line.rstrip('\n') for line in all_lines]
        f.close()
        create_translation_for_file(all_lines_split, folder_path_to_save+"/"+name_of_translation)


In [ ]:
directories = list_directories(directory_path_dataset)
directories = directories[:len(directories)]
directories = [direct for direct in directories if "twitter" not in direct and ".ipynb" not in direct and "opus" not in direct]
directories.append(directory_path_dataset + "/twitter/ksaembassyrou")
print(len(directories), directories)
#directories = [directories[-2]]

Directories in /content/drive/MyDrive/disertatie/dataset :
9 ['/content/drive/MyDrive/disertatie/dataset/drepturile_copilului', '/content/drive/MyDrive/disertatie/dataset/drepturile_omului', '/content/drive/MyDrive/disertatie/dataset/conventia_geneva', '/content/drive/MyDrive/disertatie/dataset/arabia_saudita_guvern', '/content/drive/MyDrive/disertatie/dataset/drepturile_economice_culturale', '/content/drive/MyDrive/disertatie/dataset/statutul_refugiatilor', '/content/drive/MyDrive/disertatie/dataset/flores+', '/content/drive/MyDrive/disertatie/dataset/from_training_dataset', '/content/drive/MyDrive/disertatie/dataset/twitter/ksaembassyrou']


In [ ]:
directories

['/content/drive/MyDrive/disertatie/dataset/drepturile_copilului',
 '/content/drive/MyDrive/disertatie/dataset/drepturile_omului',
 '/content/drive/MyDrive/disertatie/dataset/conventia_geneva',
 '/content/drive/MyDrive/disertatie/dataset/arabia_saudita_guvern',
 '/content/drive/MyDrive/disertatie/dataset/drepturile_economice_culturale',
 '/content/drive/MyDrive/disertatie/dataset/statutul_refugiatilor',
 '/content/drive/MyDrive/disertatie/dataset/flores+',
 '/content/drive/MyDrive/disertatie/dataset/from_training_dataset',
 '/content/drive/MyDrive/disertatie/dataset/twitter/ksaembassyrou']

In [ ]:
create_all_missing_folders(directories)

In [ ]:
read_files_and_create_translations(directories)

Creating translations for  /content/drive/MyDrive/disertatie/dataset/drepturile_copilului/ar.txt


100%|██████████| 217/217 [02:30<00:00,  1.44it/s]


The translation is saved at  /content/drive/MyDrive/disertatie/translate/NLLB_TRAINED/drepturile_copilului/nllb.txt
Creating translations for  /content/drive/MyDrive/disertatie/dataset/drepturile_omului/ar.txt


100%|██████████| 79/79 [00:38<00:00,  2.06it/s]


The translation is saved at  /content/drive/MyDrive/disertatie/translate/NLLB_TRAINED/drepturile_omului/nllb.txt
Creating translations for  /content/drive/MyDrive/disertatie/dataset/conventia_geneva/ar.txt


...

The translation is saved at  /content/drive/MyDrive/disertatie/translate/NLLB_TRAINED/flores+/nllb.txt
Creating translations for  /content/drive/MyDrive/disertatie/dataset/from_training_dataset/ar.txt


100%|██████████| 2100/2100 [13:01<00:00,  2.69it/s]


The translation is saved at  /content/drive/MyDrive/disertatie/translate/NLLB_TRAINED/from_training_dataset/nllb.txt
Creating translations for  /content/drive/MyDrive/disertatie/dataset/twitter/ksaembassyrou/ar.txt


100%|██████████| 86/86 [02:58<00:00,  2.07s/it]

The translation is saved at  /content/drive/MyDrive/disertatie/translate/NLLB_TRAINED/twitter/ksaembassyrou/nllb.txt


In [ ]:
from google.colab import runtime
runtime.unassign()

In [ ]:
import pandas as pd

ds = pd.read_csv("/content/drive/MyDrive/disertatie/dataset/flores.csv")
all_to_translate = list(ds["ar"])
create_translation_for_file(all_to_translate, directory_path_dataset_to_save+"/FLORES_nllb.txt")

100%|██████████| 1012/1012 [11:20<00:00,  1.49it/s]

The translation is saved at
